In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import CNN_Clasificator
from get_embedded_data import get_data_glove_CNN, get_data_BERT_MLP

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warn

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
# max_len_sentence = 0
# pos = 0
# for i, sentence in enumerate(list_of_words):
#     max_len_sentence, pos = (len(sentence), i) if len(sentence)>max_len_sentence else (max_len_sentence, pos)
# max_len_sentence, pos

In [4]:
# import nltk
# from nltk.tokenize import word_tokenize
# from gensim.models import Word2Vec
# # nltk.download('punkt')

# import gensim.downloader as api
# corpus = api.load('text8')

# with open('data_set.csv', 'r', encoding='utf-8') as dh:
#     list_of_words = []
#     list_of_targets = []
#     for i, line in enumerate(dh):
#         if i > 0:
#             line = line.strip()
#             line = line.split('@')
#             line[-1] = word_tokenize(line[-1].lower())
#             list_of_words.append(line[-1])
#             list_of_targets.append(line[1])
#     dh.close()

# emb_dim = 100

# model = Word2Vec(corpus, min_count=1)
# model.build_vocab(list_of_words, update=True)
# model.train(list_of_words, total_examples=model.corpus_count, epochs=20)
# model.save('word2vec_100d')

# # model = Word2Vec.load('word2vec/word2vec_100d')

# # Access embeddings
# word_embeddings = model.wv
# print(model.wv.most_similar("folly"))
# print(word_embeddings['superstitious'])


In [5]:
# word2vec_embeddings = []
# for sentence in list_of_words:
#     emb_sentence = np.empty((100,0))
#     for word in sentence:
#         emb_sentence = np.hstack((emb_sentence, np.reshape(model.wv[word], (100, 1))))
#     word2vec_embeddings.append(torch.from_numpy(emb_sentence))

In [6]:
# import torchtext
# import torchtext.vocab

# glove = torchtext.vocab.GloVe(name="6B", dim=100)

In [7]:
# list_of_inc = []
# for i, sen in enumerate(list_of_words):
#     counter = 0
#     for j, wor in enumerate(sen):
#         if wor not in glove:
#             counter += 1
#     if counter >= 1: list_of_inc.append((i, counter))
# print(len(list_of_inc))
# list_of_inc

In [8]:
# abc = []
# for i, sen in enumerate(list_of_words):
#     for j, wor in enumerate(sen):
#             if wor not in glove: abc.append((list_of_targets[i],i, j, wor))

In [9]:
# miss_by_aut=[]
# i = 0
# inc = 0
# prev_aut = list_of_inc[0][0]
# for aut , _, _ in list_of_inc:
#     if aut == prev_aut:
#         inc += 1
#     else:
#         miss_by_aut.append([prev_aut, inc])
#         prev_aut = aut
#         i+=1
#         inc = 0
# miss_by_aut.append([aut, inc])
# miss_by_aut

In [10]:
# import nltk

In [11]:
# nltk.download()

In [12]:
dataloader = get_data_BERT_MLP(20, device)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
network = CNN_Clasificator().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [14]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

print('Finished Training')

RuntimeError: Input type (float) and bias type (double) should be the same